<a href="https://colab.research.google.com/github/harveyrutland/swarm_routines/blob/main/swarmconfig_multiagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Demo

 

1.   Red triangle = boat
2.   yellow region = view region
3. blue dot = shark fin





script 2 (underdevelopment)

In [150]:
!ln -s /content/drive/MyDrive/Disstertation/swarm_boatproperties.ipynb swarm_boatproperties.ipynb 
%run swarm_boatproperties.ipynb


ln: failed to create symbolic link 'swarm_boatproperties.ipynb': File exists


In [151]:
from itertools import filterfalse
import numpy as np
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
from matplotlib import animation, rc
from IPython.display import HTML
import math
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from math import atan2, degrees, radians
from numpy import linalg as LA
from scipy.spatial import distance
import os
import sys




def init():
    return ()


def get_key_from_value(d, val):
    keys = [k for k, v in d.items() if v == val]
    if keys:
        return keys[0]
    return None


def get_angle(point_1, point_2): 
    angle = atan2(point_1[1] - point_2[1], point_1[0] - point_2[0])
    return angle






def return_dist(boatx, boaty, leaderx, leadery):
  x = [boatx, leaderx]
  y = [boaty, leadery]
  delta_x = abs(boatx - leaderx)
  delta_y = abs(boaty - leadery)
  hypoteneus = math.sqrt(delta_x**2 + delta_y**2)
  return hypoteneus







class AddShark:
  #shark
   def __init__(self):
    self.count = 0
    start = [0],[random.uniform(-1, 0)]
    end = [30],[random.uniform(-1,0)]
    N = 211
    self.frame_count = 100
    self.targetpath = np.linspace(start, end, N)
    count_interval = (N-11)/100
    self.count_interval = int(count_interval)
    
  
   def plot_shark(self, count=0, angle=0):
     x_tail = self.targetpath[count][0]
     y_tail = self.targetpath[count][1]
     x_fin = self.targetpath[count+5][0]
     y_fin = self.targetpath[count+5][1]
     x_mouth = self.targetpath[count+10][0]
     y_mouth = self.targetpath[count+10][1]

     tail = [x_tail, x_fin], [y_tail, y_fin]
     fin = x_fin, y_fin
     mouth = [x_fin, x_mouth], [y_fin, y_mouth]
     ax.plot([x_fin, x_mouth], [y_fin, y_mouth], 'bo', linestyle="--", marker=(3, 0, angle))
     ax.plot(x_fin, y_fin, 'bo')
     ax.plot([x_tail, x_fin], [y_tail, y_fin], 'bo', linestyle="--", marker=(3, 0, angle))
     return x_fin, y_fin, count






# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
fig.set_size_inches(15, 6)
plt.close()
vid_number = 0
shark_lost_report = False
first_loop = True
boat_seen = False 
shark_seen = False 
leader_assigned = False
follower_assigned = False
leader_follow_shark = False
leader_number = None
swarm_dist_to_leader = None
follower_boat = None
pause = False
travel_count = 0
boat_seen = None
swarm_position_set = False 
perp_direction_set = False
anchor_position_coords = []
saving_file = None









def animate(i):
  
  global boat_seen
  global shark_seen
  global boats
  global leader_number
  global sight_of_shark
  global leader_follow_shark
  global leader_assigned, follower_assigned
  global leader_coords, follow_coords, swarm_coords
  global swarm_dist_to_leader
  global follower_boat
  global shark_lost_report
  global first_loop
  global travel_count
  global saving_file  
  global boat_seen
  global swarm_position_set
  global perp_direction_set
  global anchor_position_coords
  global anchor1_set
  global anchor_number
  global follower_conditions_met




  setEnv.clear_frames()
  setEnv.add_start_box()
  x_fin, y_fin, count = addShark.plot_shark(addShark.count, 0)




  if first_loop == True:
    count = 0
  if count == 200:

    first_loop = True
    print('shark lost in process', shark_lost_report)
    print('leader assigned to', leader_number)
    print('boat seen', boat_seen)
    print('-------------------')
  
      



     

 
  swarm_dist_to_leader = {}
  for boat in boats:


    boat.set_rotation(x_fin, y_fin)
    seen, boatnumber, coords = boat.plot_boats(x_fin, y_fin, (boats.index(boat)+1))
    leader_number, leader_assigned = leaderFollower.set_leader(seen, leader_assigned, boatnumber)

        
    if leader_assigned == True:
      if  boats.index(boat)+1 == leader_number:
        shark_lost_report, boat_seen, perp_direction_set = boat.follow_shark(x_fin, y_fin)
        leader_coords = coords
       
       
    if  boats.index(boat)+1 != leader_number and leader_assigned == True and follower_assigned == False:
      return_dist(coords[0], coords[1], leader_coords[0], leader_coords[1])
      swarm_dist_to_leader[boatnumber] = coords
      
      if len(swarm_dist_to_leader) == (len(boats) - 1):
        # swarm_dists = swarm_dist_to_leader.values()
        lowest_dist = min(swarm_dist_to_leader.values())
        follower_boat = get_key_from_value(swarm_dist_to_leader , lowest_dist)
        follower_assigned = True

    if  boats.index(boat)+1 != leader_number and follower_assigned == True:
      if  boats.index(boat)+1 == follower_boat:
        boat.check_search_state(x_fin, y_fin)
        follow_coords = boat.follow_leader(leader_coords[0], leader_coords[1])
        if follow_coords != None:
          print('follower coords:', follow_coords)
          follower_conditions_met = True







    if swarm_position_set == False and perp_direction_set == True and follower_conditions_met == True:
      travel_count += 1
      if travel_count == 100 and swarm_position_set == False: 
        gap = 9
        anchor_position_coords = leaderFollower.triangulate_anchor(leader_coords, follow_coords, gap)
        print('**** anchor set ****', anchor_position_coords)
        swarm_position_set  = True 
        print('setting anchor location')
        
    if swarm_position_set == True and boats.index(boat)+1 != leader_number and boats.index(boat)+1 != follower_boat:
      if anchor1_set == False:
        anchor_number = boats.index(boat)+1
        boat.initaite_swarm(anchor_position_coords[0], anchor_position_coords[1], leader_coords[0], leader_coords[1])
        anchor1_set = True

      elif boats.index(boat)+1 == anchor_number:
        reset_anchor = boat.initaite_swarm(anchor_position_coords[0], anchor_position_coords[1], leader_coords[0], leader_coords[1]) 
        if reset_anchor  == True:
          gap = 5
          anchor_position_coords = leaderFollower.triangulate_anchor(leader_coords, follow_coords, gap)
          boat.check_search_state(x_fin, y_fin)
        
  



   




    
    
    
      
      
      
    

  
  addShark.count += addShark.count_interval     
  return ()



while vid_number <= 50:
  print('-------------------')
  print('running loop, vid number is', vid_number+1)




  vid_number += 1  
  
  addShark = AddShark()
  setEnv = SetEnv()
  leaderFollower = LeaderFollower()

  Boat1 = AddBoats(setEnv)
  Boat2 = AddBoats(setEnv)
  Boat3 = AddBoats(setEnv)
  Boat4 = AddBoats(setEnv)
  Boat5 = AddBoats(setEnv)

  boats = [Boat1, Boat2, Boat3 , Boat4, Boat5]


  setEnv.clear_frames()
  setEnv.add_start_box()




  first_loop = False
  shark_lost_report = False
  boat_seen = False 
  shark_seen = False 
  leader_assigned = False
  follower_assigned = False
  leader_follow_shark = False
  leader_number = None
  swarm_dist_to_leader = None
  follower_boat = None
  pause = False
  follow_coords = None
  travel_count = 0

  swarm_position_set = False 
  perp_direction_set = False
  anchor_position_coords = []
  anchor1_set = False
  anchor_number = None
  follower_conditions_met = False


  







  anim = animation.FuncAnimation(fig, animate, init_func=init, frames= addShark.frame_count+1, interval=100, blit=True)
  rc('animation', html='jshtml')
  anim


  writervideo = animation.FFMpegWriter(fps=60)
  anim.save('/content/drive/MyDrive/Disstertation/test' + str(vid_number) + '.avi', writer=writervideo)
    




-------------------
running loop, vid number is 1
follower coords: [5.041548002036673, 2.2435300592003133]
follower coords: [5.041548002036673, 2.2435300592003133]
follower coords: [5.041548002036673, 2.2435300592003133]
follower coords: [5.041548002036673, 2.2435300592003133]
follower coords: [5.041548002036673, 2.2435300592003133]
follower coords: [5.041548002036673, 2.2435300592003133]
follower coords: [5.280052225886904, 2.07531140374964]
follower coords: [5.52712859862072, 1.9618854347137553]
follower coords: [5.784542838127396, 1.8902840555819418]
follower coords: [6.04650340666637, 1.8494521039419451]
follower coords: [6.31434808222327, 1.8321593825430103]
follower coords: [6.584811611446618, 1.8321859591847478]
follower coords: [6.8584613620446575, 1.812414705826162]
follower coords: [7.138489745202927, 1.8115557000640068]
follower coords: [7.4157924186234645, 1.823478225392965]
follower coords: [7.696020011431499, 1.8122240413817798]
follower coords: [7.980113667300014, 1.8175

Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x7eff2d479050>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/transforms.py", line 177, in <lambda>
    ref = weakref.ref(self, lambda ref, sid=id(self),
KeyboardInterrupt


follower coords: [9.679710738738788, 1.636628977476282]
step number: 22
adapted leader follower anlge 3.3920524389099853


CalledProcessError: ignored